# 2023년도 해안 쓰레기 기준 - 60개 정점의 쓰레기 양과 정점 위치 데이터
가공전 데이터 : D:\marideb\code\geo_locations_google.csv  
(가공) 사용 데이터 : D:\marideb\code\해양쓰레기_위치통합데이터.csv



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, MarkerCluster
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

# 통합 데이터 불러오기
integrated_data = pd.read_csv('해양쓰레기_위치통합데이터.csv')

# 데이터 구조 확인
print("통합 데이터 구조:")
print(integrated_data.head())
print(f"통합 데이터 크기: {integrated_data.shape}")
print("\n데이터 기본 정보:")
print(integrated_data.info())
print("\n기술 통계량:")
print(integrated_data.describe())

In [ ]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
df = pd.read_csv('해양쓰레기_위치통합데이터.csv', encoding='utf-8')

# 6개 차수의 전체 조사량 데이터와 날짜 입력
survey_rounds = {
    '1차': {'개수': 15014, '무게': 1364.9, '날짜': '2023-01-31'},
    '2차': {'개수': 11051, '무게': 473.4, '날짜': '2023-03-31'},
    '3차': {'개수': 11286, '무게': 659.5, '날짜': '2023-05-31'},
    '4차': {'개수': 13219, '무게': 340.9, '날짜': '2023-07-31'},
    '5차': {'개수': 15582, '무게': 490.4, '날짜': '2023-09-30'},
    '6차': {'개수': 12324, '무게': 825.9, '날짜': '2023-11-30'}
}

# 각 정점의 총 쓰레기 개수와 무게 비율 계산
total_count = df['개수'].sum()
total_weight = df['무게_kg'].sum()

print(f"CSV 파일 내 총 쓰레기 개수: {total_count}")
print(f"CSV 파일 내 총 쓰레기 무게: {total_weight}kg")

# 각 차수별 전체 개수와 무게
total_survey_count = sum(round_data['개수'] for round_data in survey_rounds.values())
total_survey_weight = sum(round_data['무게'] for round_data in survey_rounds.values())

print(f"모든 차수 합산 총 쓰레기 개수: {total_survey_count}")
print(f"모든 차수 합산 총 쓰레기 무게: {total_survey_weight}kg")

# 결과 데이터프레임 준비 (각 정점별, 차수별 데이터를 저장할 구조)
result_df = []

# 각 정점별로 차수별 데이터 추산
for _, row in df.iterrows():
    station = row['정점명']
    count = row['개수']
    weight = row['무게_kg']
    
    # 정점의 개수/무게가 전체에서 차지하는 비율
    count_ratio = count / total_count if total_count > 0 else 0
    weight_ratio = weight / total_weight if total_weight > 0 else 0
    
    # 각 차수별로 정점의 쓰레기 개수와 무게 추산
    for round_num, round_data in survey_rounds.items():
        # 이 차수의 전체 개수와 무게
        round_total_count = round_data['개수']
        round_total_weight = round_data['무게']
        
        # 비례 배분 - 해당 정점의 차수별 추정치
        estimated_count = round(count_ratio * round_total_count)
        estimated_weight = round(weight_ratio * round_total_weight, 2)
        
        # 결과 데이터에 추가 (날짜 포함)
        result_df.append({
            '정점명': station,
            '차수': round_num,
            '날짜': round_data['날짜'],
            '추정_개수': estimated_count,
            '추정_무게_kg': estimated_weight,
            '위도': row['위도'],
            '경도': row['경도'],
            '군집': row['군집']
        })

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_df)

# 날짜 형식 변환 (필요시)
result_df['날짜'] = pd.to_datetime(result_df['날짜'])

# 결과 출력
print("\n추정 결과 (처음 10개 샘플):")
print(result_df.head(10))

# 차수별 합계 검증
print("\n차수별 합계 검증:")
for round_num, round_data in survey_rounds.items():
    round_sum = result_df[result_df['차수'] == round_num]['추정_개수'].sum()
    round_weight_sum = result_df[result_df['차수'] == round_num]['추정_무게_kg'].sum()
    print(f"{round_num} 추정 합계 - 개수: {round_sum} (원본: {round_data['개수']}), 무게: {round_weight_sum:.2f}kg (원본: {round_data['무게']}kg)")

# 결과를 CSV 파일로 저장
result_df.to_csv('해양쓰레기_차수별_추정결과.csv', index=False, encoding='utf-8')
print("\n추정 결과가 '해양쓰레기_차수별_추정결과.csv' 파일로 저장되었습니다.")

각 정점의 위치와, 1차 ~6차 별 추정 쓰레기 양을 정리했다. 이제 그 시기별 선박밀집도와 해류를 쓰려고 한다. 
일단 선박밀집도 데이터 분석
일단 정점에서 가장 가까운 그리드명을 찾는 코드를 짜자


In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    두 지점의 위경도 좌표를 이용하여 거리를 계산하는 함수 (단위: km)
    """
    # 위경도를 라디안으로 변환
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # 하버사인 공식
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # 지구 반경 (km)
    
    return c * r

def find_nearest_grid(station_data, grid_data):
    """
    각 정점에서 가장 가까운 그리드를 찾는 함수
    
    Parameters:
    - station_data: 정점 데이터 (DataFrame)
    - grid_data: 그리드 데이터 (DataFrame)
    
    Returns:
    - 각 정점별 가장 가까운 그리드 정보 (DataFrame)
    """
    result = []
    
    for _, station in station_data.iterrows():
        station_name = station['정점명']
        station_lat = station['위도']
        station_lon = station['경도']
        
        min_distance = float('inf')
        nearest_grid = None
        
        for _, grid in grid_data.iterrows():
            grid_name = grid['Grid_ID']  # 그리드 이름 컬럼명 (실제 데이터에 맞게 수정 필요)
            grid_lat = grid['Latitude']    # 위도 컬럼명 (실제 데이터에 맞게 수정 필요)
            grid_lon = grid['Longitude']   # 경도 컬럼명 (실제 데이터에 맞게 수정 필요)
            
            distance = haversine(station_lon, station_lat, grid_lon, grid_lat)
            
            if distance < min_distance:
                min_distance = distance
                nearest_grid = grid_name
        
        result.append({
            '정점명': station_name,
            '정점_위도': station_lat,
            '정점_경도': station_lon,
            '가장_가까운_그리드': nearest_grid,
            '거리_km': min_distance
        })
    
    return pd.DataFrame(result)

# 메인 실행 코드
def main():
    # 정점 데이터 로드
    station_data = pd.read_csv('marideb_location.csv')
    
    # 그리드 데이터 로드 (실제 선박밀집도 그리드 데이터 파일명으로 수정 필요)
    grid_data = pd.read_csv('D:/marideb/code/cleaned_sden_202502200100_grid3.csv')  
    
    # 각 정점에서 가장 가까운 그리드 찾기
    result = find_nearest_grid(station_data, grid_data)
    
    # 결과 저장
    result.to_csv('station_nearest_grid.csv', index=False, encoding='utf-8-sig')
    print(f"총 {len(result)}개 정점의 가장 가까운 그리드 정보를 저장했습니다.")
    
    return result

if __name__ == "__main__":
    main()

정점별 가장 가까운 그리드 : code\station_nearest_grid.csv  
  

이후 조사날짜 이전 30일 선박밀집도 데이터를 처리하는 코드를 짜야지.


In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

def test_load_single_file():
    """
    단일 선박밀집도 파일 로드 테스트
    """
    # 파일 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'
    
    # 임의의 날짜 선택 (예: 2023년 1월 1일)
    date_formatted = '20230101'
    file_name = f"sden_{date_formatted}0100_grid3.csv"
    file_path = os.path.join(base_path, file_name)
    
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        print(f"파일이 존재하지 않습니다: {file_path}")
        
        # 디렉토리의 일부 파일 목록 출력
        print("\n디렉토리 파일 목록:")
        try:
            files = os.listdir(base_path)
            for i, f in enumerate(files[:10]):  # 처음 10개만 출력
                print(f"  - {f}")
            if len(files) > 10:
                print(f"  - ... 외 {len(files)-10}개 파일")
        except Exception as e:
            print(f"디렉토리 접근 오류: {str(e)}")
        
        # 다른 형식의 파일명 시도
        alternate_patterns = [
            f"sden_{date_formatted}0000_grid3.csv",
            f"sden_{date_formatted}_grid3.csv",
            f"sden_{date_formatted}.csv"
        ]
        
        for pattern in alternate_patterns:
            alt_path = os.path.join(base_path, pattern)
            if os.path.exists(alt_path):
                print(f"대체 파일을 찾았습니다: {pattern}")
                file_path = alt_path
                break
        
        # 날짜 패턴을 포함하는 파일 찾기
        if not os.path.exists(file_path):
            print("\n날짜 패턴을 포함하는 파일 찾기:")
            for f in files:
                if date_formatted in f and f.endswith('.csv'):
                    print(f"날짜 패턴 포함 파일: {f}")
                    file_path = os.path.join(base_path, f)
                    break
    
    # 파일 로드 시도
    if os.path.exists(file_path):
        try:
            print(f"\n파일 로드 시도: {file_path}")
            data = pd.read_csv(file_path)
            print(f"파일 로드 성공: {data.shape[0]} 행 x {data.shape[1]} 열")
            print(f"컬럼명: {data.columns.tolist()}")
            print("\n첫 5개 행:")
            print(data.head(5))
            
            # 격자번호 및 밀집도 컬럼 확인
            if '격자번호' in data.columns:
                print(f"\n격자번호 샘플: {data['격자번호'].head(3).tolist()}")
            else:
                print("\n'격자번호' 컬럼이 없습니다. 실제 컬럼명을 확인하세요.")
            
            if '밀집도(%)' in data.columns:
                print(f"밀집도(%) 통계: 평균={data['밀집도(%)'].mean():.2f}, 최대={data['밀집도(%)'].max():.2f}")
            else:
                print("'밀집도(%)' 컬럼이 없습니다. 실제 컬럼명을 확인하세요.")
                
            return data
        except Exception as e:
            print(f"파일 로드 오류: {str(e)}")
    else:
        print("사용 가능한 파일을 찾을 수 없습니다.")
    
    return None

def test_nearest_grid():
    """
    정점에서 가장 가까운 그리드 찾기 테스트
    """
    try:
        # 정점 데이터 로드
        station_data = pd.read_csv('marideb_location.csv')
        print(f"정점 데이터 로드 성공: {station_data.shape[0]} 행")
        
        # 그리드 데이터 로드
        ship_data = test_load_single_file()
        
        if ship_data is None:
            print("그리드 데이터 로드 실패")
            return
            
        # 첫 번째 정점에 대해 테스트
        if len(station_data) > 0:
            test_station = station_data.iloc[0]
            print(f"\n테스트 정점: {test_station['정점명']}")
            print(f"위치: 위도={test_station['위도']}, 경도={test_station['경도']}")
            
            # 모든 그리드와의 거리 계산 테스트
            from math import radians, cos, sin, asin, sqrt
            
            def haversine(lon1, lat1, lon2, lat2):
                lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
                dlon = lon2 - lon1 
                dlat = lat2 - lat1 
                a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
                c = 2 * asin(sqrt(a)) 
                r = 6371  # 지구 반경 (km)
                return c * r
            
            # 각 그리드와의 거리 계산
            distances = []
            
            for _, grid in ship_data.iterrows():
                try:
                    grid_name = grid['격자번호']
                    lat_lon = grid['위경도'].split(', ')
                    grid_lat = float(lat_lon[0])
                    grid_lon = float(lat_lon[1])
                    
                    distance = haversine(test_station['경도'], test_station['위도'], 
                                        grid_lon, grid_lat)
                    
                    distances.append({
                        '격자번호': grid_name,
                        '위도': grid_lat,
                        '경도': grid_lon,
                        '거리_km': distance
                    })
                except Exception as e:
                    print(f"그리드 거리 계산 오류: {str(e)}")
            
            # 거리별로 정렬
            distances_df = pd.DataFrame(distances)
            distances_df = distances_df.sort_values('거리_km')
            
            print("\n가장 가까운 그리드 5개:")
            print(distances_df.head(5))
            
            return True
    except Exception as e:
        print(f"테스트 오류: {str(e)}")
    
    return False

# 테스트 실행
if __name__ == "__main__":
    print("=== 선박밀집도 데이터 처리 테스트 ===")
    test_nearest_grid()

=== 선박밀집도 데이터 처리 테스트 ===
정점 데이터 로드 성공: 60 행

파일 로드 시도: D:\marideb\code\sden_2023_lv3\sden_202301010100_grid3.csv
파일 로드 성공: 3658 행 x 21 열
컬럼명: ['격자번호', '위경도', '교통량(척)', '밀집도(%)', '어선', '여객선', '화물선', '유조선', '예인선', '수상레저기구', '기타선', '1t미만', '1t ~ 2t', '2t ~ 3t', '3t ~ 5t', '5t ~ 10t', '10t ~ 50t', '50t ~ 100t', '100t ~ 500t', '500t이상', '미상']

첫 5개 행:
          격자번호              위경도  교통량(척)  밀집도(%)   어선  여객선  화물선  유조선  예인선  \
0  GR3_G3E12_S  34.825, 128.425     194   17.19  183    0    1    1    0   
1  GR3_G3B33_P  35.075, 129.025     118   35.47   58    1   24    2   14   
2  GR3_G3B33_L  35.125, 129.075     102  100.00    4    2   27   28   21   
3  GR3_G3B14_W  35.525, 129.375      78   67.85    3    2   11   10   25   
4  GR3_F4H42_A  34.475, 127.775      75    5.08   67    0    2    0    4   

   수상레저기구  ...  1t미만  1t ~ 2t  2t ~ 3t  3t ~ 5t  5t ~ 10t  10t ~ 50t  \
0       0  ...     0        0        0        7         5        102   
1       1  ...     0        0        0        0   

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def process_ship_density_data(timestep_data, nearest_grid_data, ship_density_data):
    """
    각 조사 차수와 날짜에 대해 이전 30일간의 선박밀집도 데이터 처리
    
    Parameters:
    - timestep_data: 조사 차수 및 날짜 데이터 (DataFrame)
    - nearest_grid_data: 각 정점별 가장 가까운 그리드 정보 (DataFrame)
    - ship_density_data: 선박밀집도 데이터 (Dictionary 형태, key: 날짜, value: 해당 날짜의 DataFrame)
    
    Returns:
    - 각 정점별 30일 선박밀집도 정보 (DataFrame)
    """
    result = []
    
    # 정점명과 가장 가까운 그리드 매핑 생성
    grid_mapping = dict(zip(nearest_grid_data['정점명'], nearest_grid_data['가장_가까운_그리드']))
    
    # 각 차수별, 정점별로 처리
    for _, row in timestep_data.iterrows():
        station_name = row['정점명']
        round_num = row['차수']
        survey_date = datetime.strptime(row['날짜'], '%Y-%m-%d')  # 날짜 형식은 실제 데이터에 맞게 조정 필요
        
        # 해당 정점의 가장 가까운 그리드 찾기
        if station_name not in grid_mapping:
            print(f"경고: '{station_name}' 정점의 가장 가까운 그리드 정보가 없습니다.")
            continue
            
        nearest_grid = grid_mapping[station_name]
        
        # 이전 30일 날짜 리스트 생성
        date_list = [(survey_date - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 31)]
        
        # 30일간의 선박밀집도 데이터 수집
        density_values = []
        
        for date in date_list:
            if date in ship_density_data:
                # 해당 날짜의 선박밀집도 데이터에서 해당 그리드의 값 찾기
                # 실제 데이터 구조에 맞게 수정
                grid_id_col = '격자번호'
                density_col = '밀집도(%)'
                
                grid_density = ship_density_data[date].loc[
                    ship_density_data[date][grid_id_col] == nearest_grid, 
                    density_col
                ].values
                
                if len(grid_density) > 0:
                    density_values.append(grid_density[0])
                else:
                    print(f"경고: {date} 날짜에 {nearest_grid} 그리드의 선박밀집도 데이터가 없습니다.")
                    density_values.append(np.nan)
            else:
                print(f"경고: {date} 날짜의 선박밀집도 데이터가 없습니다.")
                density_values.append(np.nan)
        
        # 30일 평균 선박밀집도 계산
        avg_density = np.nanmean(density_values) if len(density_values) > 0 else np.nan
        max_density = np.nanmax(density_values) if len(density_values) > 0 else np.nan
        
        result.append({
            '정점명': station_name,
            '차수': round_num,
            '조사날짜': survey_date.strftime('%Y-%m-%d'),
            '30일_평균_선박밀집도': avg_density,
            '30일_최대_선박밀집도': max_density,
            '데이터_포함일수': sum(~np.isnan(density_values)),
            '가장_가까운_그리드': nearest_grid
        })
    
    return pd.DataFrame(result)

def load_ship_density_data(base_path, date_list):
    """
    여러 날짜의 선박밀집도 데이터를 로드하는 함수
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - date_list: 로드할 날짜 리스트 (YYYY-MM-DD 형식)
    
    Returns:
    - 날짜별 선박밀집도 데이터 (Dictionary)
    """
    import os
    ship_density_data = {}
    
    for date in date_list:
        try:
            # 날짜 형식 변환 (YYYY-MM-DD -> YYYYMMDD)
            date_obj = datetime.strptime(date, '%Y-%m-%d')
            date_formatted = date_obj.strftime('%Y%m%d')
            
            # 파일명 형식: sden_YYYYMMDDHHMM_grid3.csv
            # 일단 해당 날짜의 0100(01시) 데이터를 사용
            file_name = f"sden_{date_formatted}0100_grid3.csv"
            file_path = os.path.join(base_path, file_name)
            
            # 파일이 존재하는지 확인
            if not os.path.exists(file_path):
                print(f"경고: 파일이 존재하지 않습니다: {file_path}")
                continue
                
            # 데이터 로드
            data = pd.read_csv(file_path)
            ship_density_data[date] = data
            
            # 진행 상황 표시 (20개마다)
            if len(ship_density_data) % 20 == 0:
                print(f"정보: {len(ship_density_data)}개 날짜의 데이터를 로드했습니다...")
                
        except Exception as e:
            print(f"경고: {date} 날짜의 선박밀집도 데이터 로드 실패: {str(e)}")
    
    print(f"정보: 총 {len(ship_density_data)}개 날짜의 선박밀집도 데이터를 로드했습니다.")
    return ship_density_data

# 메인 실행 코드
def main():
    # 조사 차수 및 날짜 데이터 로드
    timestep_data = pd.read_csv('marideb_timestep.csv')
    
    # 정점별 가장 가까운 그리드 정보 로드 (find_nearest_grid 함수로 생성한 파일)
    nearest_grid_data = pd.read_csv('station_nearest_grid.csv')
    
    # 조사 날짜의 고유 목록 생성
    unique_survey_dates = timestep_data['날짜'].unique()
    
    # 모든 조사에 필요한 이전 30일 날짜 목록 생성
    all_required_dates = set()
    for date_str in unique_survey_dates:
        survey_date = datetime.strptime(date_str, '%Y-%m-%d')  # 날짜 형식은 실제 데이터에 맞게 조정 필요
        for i in range(1, 31):
            prev_date = (survey_date - timedelta(days=i)).strftime('%Y-%m-%d')
            all_required_dates.add(prev_date)
    
    # 선박밀집도 데이터 로드 (기본 경로는 실제 데이터 위치에 맞게 조정 필요)
    base_path = r'D:\marideb\code\sden_2023_lv3'  # raw 문자열 사용
    ship_density_data = load_ship_density_data(base_path, list(all_required_dates))
    
    # 30일 선박밀집도 데이터 처리
    result = process_ship_density_data(timestep_data, nearest_grid_data, ship_density_data)
    
    # 결과 저장
    result.to_csv('station_30days_ship_density.csv', index=False, encoding='utf-8-sig')
    print(f"총 {len(result)}개 정점-차수의 30일 선박밀집도 정보를 저장했습니다.")
    
    return result

if __name__ == "__main__":
    main()

정보: 20개 날짜의 데이터를 로드했습니다...
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303260100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303290100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303130100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303300100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303120100_grid3.csv
정보: 40개 날짜의 데이터를 로드했습니다...
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303280100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303270100_grid3.csv
정보: 60개 날짜의 데이터를 로드했습니다...
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303110100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303230100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303220100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303170100_grid3.csv
경고: 파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202303150100_grid3

C:\Users\TAN\AppData\Local\Temp\ipykernel_13156\4209507410.py:63: RuntimeWarning: Mean of empty slice
  avg_density = np.nanmean(density_values) if len(density_values) > 0 else np.nan
C:\Users\TAN\AppData\Local\Temp\ipykernel_13156\4209507410.py:64: RuntimeWarning: All-NaN axis encountered
  max_density = np.nanmax(density_values) if len(density_values) > 0 else np.nan


경고: 2023-05-10 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-05-08 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-05-07 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-05-04 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-05-01 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-11-29 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-11-28 날짜에 GR3_F2K14_M 그리드의 선박밀집도 데이터가 없습니다.
경고: 2023-01-13 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-30 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-29 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-28 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-27 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-26 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-25 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-24 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-23 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-22 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-21 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-20 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-19 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-18 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-17 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-16 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-15 날짜의 선박밀집도 데이터가 없습니다.
경고: 2023-03-14 날짜

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap
import os
import glob
from datetime import datetime, timedelta


# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')  # Windows의 경우
plt.rc('axes', unicode_minus=False)

def create_ship_density_animation(base_path, output_gif='ship_density_animation.gif', num_days=7):
    """
    여러 날짜의 선박밀집도 데이터를 애니메이션으로 시각화
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - output_gif: 출력 GIF 파일 경로
    - num_days: 애니메이션에 포함할 날짜 수
    """
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 날짜 리스트 생성
    dates = [start_date + timedelta(days=i) for i in range(num_days)]
    
    # 데이터 파일 리스트 생성
    file_paths = []
    
    for date in dates:
        date_formatted = date.strftime('%Y%m%d')
        file_pattern = os.path.join(base_path, f"sden_{date_formatted}0100_grid3.csv")
        
        if os.path.exists(file_pattern):
            file_paths.append((date, file_pattern))
    
    if not file_paths:
        print("애니메이션으로 만들 데이터 파일을 찾을 수 없습니다.")
        return
    
    print(f"{len(file_paths)}개 날짜의 데이터를 애니메이션으로 만듭니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    # 모든 데이터에서 밀집도 최대값 찾기 (색상 스케일 통일을 위해)
    max_density = 0
    
    for _, file_path in file_paths:
        data = pd.read_csv(file_path)
        max_value = data['밀집도(%)'].max()
        if max_value > max_density:
            max_density = max_value
    
    # 애니메이션 설정
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 애니메이션 함수
    def animate(i):
        ax.clear()
        date, file_path = file_paths[i]
        
        try:
            data = pd.read_csv(file_path)
            
            # 산점도 그리기
            x = []  # 경도
            y = []  # 위도
            s = []  # 마커 크기 (교통량)
            c = []  # 색상 (밀집도)
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    if grid_id in grid_locations:
                        lat, lon = grid_locations[grid_id]
                        y.append(lat)
                        x.append(lon)
                        s.append(traffic / 5)  # 교통량에 따른 마커 크기 조정
                        c.append(density)
                except:
                    continue
            
            scatter = ax.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                                 vmin=0, vmax=max_density, edgecolors='k', linewidths=0.5)
            
            # 제목 및 레이블
            ax.set_title(f'선박밀집도 ({date.strftime("%Y-%m-%d")})')
            ax.set_xlabel('경도')
            ax.set_ylabel('위도')
            
            # 컬러바 추가
            if i == 0:
                global cbar
                cbar = fig.colorbar(scatter)
                cbar.set_label('밀집도 (%)')
            
            # 한국 해안선 형태를 대략적으로 표현
            ax.set_xlim([126, 130])
            ax.set_ylim([34, 38])
            
            # 그리드 추가
            ax.grid(True, linestyle='--', alpha=0.6)
            
        except Exception as e:
            print(f"애니메이션 프레임 {i} 처리 오류: {str(e)}")
    
    # 애니메이션 생성
    ani = animation.FuncAnimation(fig, animate, frames=len(file_paths), interval=400, blit=False)
    
    # GIF로 저장
    writer = animation.PillowWriter(fps=1.0)
    ani.save(output_gif, writer=writer)
    
    print(f"애니메이션을 {output_gif}에 저장했습니다.")
    plt.close()

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    
    # 선박밀집도 애니메이션 생성 (7일간)
    create_ship_density_animation(base_path, num_days=31)

30개 날짜의 데이터를 애니메이션으로 만듭니다.
애니메이션을 ship_density_animation.gif에 저장했습니다.


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap
import os
import glob
from datetime import datetime, timedelta
import matplotlib.patches as patches

def create_ship_density_animation(base_path, station_grid_file, output_gif='ship_density_animation_JAN.gif', num_days=7):
    """
    여러 날짜의 선박밀집도 데이터를 애니메이션으로 시각화하고
    해양쓰레기 60개 정점에 해당하는 그리드를 초록색 네모로 표시
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - station_grid_file: 정점별 가장 가까운 그리드 정보 파일 경로
    - output_gif: 출력 GIF 파일 경로
    - num_days: 애니메이션에 포함할 날짜 수
    """
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 날짜 리스트 생성
    dates = [start_date + timedelta(days=i) for i in range(num_days)]
    
    # 데이터 파일 리스트 생성
    file_paths = []
    
    for date in dates:
        date_formatted = date.strftime('%Y%m%d')
        file_pattern = os.path.join(base_path, f"sden_{date_formatted}0100_grid3.csv")
        
        if os.path.exists(file_pattern):
            file_paths.append((date, file_pattern))
    
    if not file_paths:
        print("애니메이션으로 만들 데이터 파일을 찾을 수 없습니다.")
        return
    
    print(f"{len(file_paths)}개 날짜의 데이터를 애니메이션으로 만듭니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    # 정점별 가장 가까운 그리드 정보 로드
    station_grids = set()
    if os.path.exists(station_grid_file):
        station_grid_data = pd.read_csv(station_grid_file)
        station_grids = set(station_grid_data['가장_가까운_그리드'].tolist())
        print(f"해양쓰레기 정점 그리드 {len(station_grids)}개를 로드했습니다.")
    else:
        print(f"경고: 정점 그리드 파일이 존재하지 않습니다: {station_grid_file}")
    
    # 모든 데이터에서 밀집도 최대값 찾기 (색상 스케일 통일을 위해)
    max_density = 0
    
    for _, file_path in file_paths:
        data = pd.read_csv(file_path)
        max_value = data['밀집도(%)'].max()
        if max_value > max_density:
            max_density = max_value
    
    # 애니메이션 설정
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 애니메이션 함수
    def animate(i):
        ax.clear()
        date, file_path = file_paths[i]
        
        try:
            data = pd.read_csv(file_path)
            
            # 산점도 그리기
            x = []  # 경도
            y = []  # 위도
            s = []  # 마커 크기 (교통량)
            c = []  # 색상 (밀집도)
            
            # 해양쓰레기 정점 그리드 위치
            station_x = []
            station_y = []
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    if grid_id in grid_locations:
                        lat, lon = grid_locations[grid_id]
                        y.append(lat)
                        x.append(lon)
                        s.append(traffic / 5)  # 교통량에 따른 마커 크기 조정
                        c.append(density)
                        
                        # 해양쓰레기 정점 그리드 표시를 위한 좌표 저장
                        if grid_id in station_grids:
                            station_x.append(lon)
                            station_y.append(lat)
                except:
                    continue
            
            # 일반 그리드 데이터 그리기
            scatter = ax.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                                 vmin=0, vmax=max_density, edgecolors='k', linewidths=0.5)
            
            # 해양쓰레기 정점 그리드 표시 (초록색 네모)
            box_size = 0.06  # 네모 크기 조정
            for sx, sy in zip(station_x, station_y):
                # 초록색 네모 그리기
                rect = patches.Rectangle(
                    (sx - box_size/2, sy - box_size/2), box_size, box_size, 
                    linewidth=1, edgecolor='lime', facecolor='none'
                )
                ax.add_patch(rect)
            
            # 범례 추가
            if i == 0:
                # 가짜 사각형 생성 (범례용)
                legend_rect = patches.Rectangle((0, 0), 1, 1, linewidth=2, edgecolor='lime', facecolor='none')
                # 범례 추가
                ax.legend([legend_rect], ['해양쓰레기 정점 그리드'], loc='upper left')
            
            # 제목 및 레이블
            ax.set_title(f'선박밀집도 ({date.strftime("%Y-%m-%d")}) - 초록색 네모: 해양쓰레기 정점', fontsize=14)
            ax.set_xlabel('경도', fontsize=12)
            ax.set_ylabel('위도', fontsize=12)
            
            # 컬러바 추가
            if i == 0:
                global cbar
                cbar = fig.colorbar(scatter)
                cbar.set_label('밀집도 (%)')
            
            # 한국 해안선 형태를 대략적으로 표현하기 위한 좌표 범위 설정
            ax.set_xlim([126, 130])
            ax.set_ylim([34, 38])
            
            # 그리드 추가
            ax.grid(True, linestyle='--', alpha=0.6)
            
            # 현재 표시된 정점 수를 화면에 표시
            ax.text(
                0.02, 0.97, 
                f'표시된 해양쓰레기 정점 그리드: {len(station_x)}개',
                transform=ax.transAxes,
                fontsize=12,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
            )
            
        except Exception as e:
            print(f"애니메이션 프레임 {i} 처리 오류: {str(e)}")
    
    # 애니메이션 생성
    ani = animation.FuncAnimation(fig, animate, frames=len(file_paths), interval=400, blit=False)
    
    # GIF로 저장
    writer = animation.PillowWriter(fps=1)
    ani.save(output_gif, writer=writer)
    
    print(f"애니메이션을 {output_gif}에 저장했습니다.")
    plt.close()

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    station_grid_file = 'station_nearest_grid.csv'  # 정점별 가장 가까운 그리드 정보 파일
    
    # 선박밀집도 애니메이션 생성 (7일간)
    create_ship_density_animation(base_path, station_grid_file, num_days=31)

30개 날짜의 데이터를 애니메이션으로 만듭니다.
해양쓰레기 정점 그리드 60개를 로드했습니다.
애니메이션을 ship_density_animation_JAN.gif에 저장했습니다.


해양쓰레기가 많은 곳과 선박밀집도가 높은 곳이 진짜 유사한지 살펴보자- 데이터적으로  
- 단, 해류 영향도 있을 거니깐 딱 일치하지 않아도 괜찮다.
- 하류 유입 위치도 영향이 있을 거니깐. 어떤 프록시변수가 관련되어있을지 더 생각해보기.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap, Normalize
import os
import glob
from datetime import datetime, timedelta
import matplotlib.patches as patches

def create_ship_density_animation(base_path, station_grid_file, debris_data_file, output_gif='ship_density_animation_JAN_grad.gif', num_days=7):
    """
    여러 날짜의 선박밀집도 데이터를 애니메이션으로 시각화하고
    해양쓰레기 정점을 쓰레기 양에 따라 색상이 다른 네모로 표시
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - station_grid_file: 정점별 가장 가까운 그리드 정보 파일 경로
    - debris_data_file: 해양쓰레기 데이터 파일 경로
    - output_gif: 출력 GIF 파일 경로
    - num_days: 애니메이션에 포함할 날짜 수
    """
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 날짜 리스트 생성
    dates = [start_date + timedelta(days=i) for i in range(num_days)]
    
    # 데이터 파일 리스트 생성
    file_paths = []
    
    for date in dates:
        date_formatted = date.strftime('%Y%m%d')
        file_pattern = os.path.join(base_path, f"sden_{date_formatted}0100_grid3.csv")
        
        if os.path.exists(file_pattern):
            file_paths.append((date, file_pattern))
    
    if not file_paths:
        print("애니메이션으로 만들 데이터 파일을 찾을 수 없습니다.")
        return
    
    print(f"{len(file_paths)}개 날짜의 데이터를 애니메이션으로 만듭니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    # 정점별 가장 가까운 그리드 정보 로드
    station_grids = {}
    if os.path.exists(station_grid_file):
        station_grid_data = pd.read_csv(station_grid_file)
        for _, row in station_grid_data.iterrows():
            station_grids[row['정점명']] = row['가장_가까운_그리드']
        print(f"해양쓰레기 정점 그리드 {len(station_grids)}개를 로드했습니다.")
    else:
        print(f"경고: 정점 그리드 파일이 존재하지 않습니다: {station_grid_file}")
    
    # 해양쓰레기 데이터 로드
    debris_data = {}
    if os.path.exists(debris_data_file):
        debris_df = pd.read_csv(debris_data_file)
        
        # 개수 또는 무게 정보가 있는지 확인
        if '개수' in debris_df.columns:
            # 정점별 쓰레기 개수 저장
            for _, row in debris_df.iterrows():
                debris_data[row['정점명']] = {
                    '개수': row['개수'],
                    '무게_kg': row['무게_kg'] if '무게_kg' in debris_df.columns else 0
                }
            
            # 개수 범위 설정
            min_debris = debris_df['개수'].min()
            max_debris = debris_df['개수'].max()
            
            print(f"해양쓰레기 데이터 로드: 개수 범위 {min_debris}~{max_debris}")
        else:
            print("해양쓰레기 데이터에 '개수' 컬럼이 없습니다.")
    else:
        print(f"경고: 해양쓰레기 데이터 파일이 존재하지 않습니다: {debris_data_file}")
    
    # 모든 데이터에서 밀집도 최대값 찾기 (색상 스케일 통일을 위해)
    max_density = 0
    
    for _, file_path in file_paths:
        data = pd.read_csv(file_path)
        max_value = data['밀집도(%)'].max()
        if max_value > max_density:
            max_density = max_value
    
    # 애니메이션 설정
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 해양쓰레기 양에 따른 색상 정규화
    if debris_data:
        debris_amounts = [data['개수'] for data in debris_data.values()]
        min_debris = min(debris_amounts)
        max_debris = max(debris_amounts)
        debris_norm = Normalize(vmin=min_debris, vmax=max_debris)
    
    # 애니메이션 함수
    def animate(i):
        ax.clear()
        date, file_path = file_paths[i]
        
        try:
            data = pd.read_csv(file_path)
            
            # 산점도 그리기
            x = []  # 경도
            y = []  # 위도
            s = []  # 마커 크기 (교통량)
            c = []  # 색상 (밀집도)
            
            # 해양쓰레기 정점 그리드 위치 및 색상 정보
            debris_info = []
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    if grid_id in grid_locations:
                        lat, lon = grid_locations[grid_id]
                        y.append(lat)
                        x.append(lon)
                        s.append(max(traffic / 5, 10))  # 교통량에 따른 마커 크기 조정, 최소 크기 10
                        c.append(density)
                        
                        # 이 그리드에 해당하는 정점 찾기
                        for station, station_grid in station_grids.items():
                            if station_grid == grid_id:
                                # 해양쓰레기 데이터가 있는 경우
                                if station in debris_data:
                                    debris_count = debris_data[station]['개수']
                                    debris_weight = debris_data[station]['무게_kg']
                                    
                                    # 해양쓰레기 정보 저장
                                    debris_info.append({
                                        'station': station,
                                        'grid': grid_id,
                                        'lon': lon,
                                        'lat': lat,
                                        'count': debris_count,
                                        'weight': debris_weight
                                    })
                except:
                    continue
            
            # 일반 그리드 데이터 그리기
            scatter = ax.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                                 vmin=0, vmax=max_density, edgecolors='k', linewidths=0.5)
            
            # 해양쓰레기 정점 그리드 표시 (쓰레기 양에 따른 색상)
            box_size = 0.05  # 네모 크기 조정
            
            if debris_info:
                # 범례를 위한 패치 리스트
                legend_elements = []
                
                for info in debris_info:
                    # 쓰레기 양에 따라 색상 결정
                    if 'count' in info:
                        color = cm(debris_norm(info['count']))
                    else:
                        color = 'lime'  # 기본 색상
                    
                    # 네모 그리기
                    rect = patches.Rectangle(
                        (info['lon'] - box_size/2, info['lat'] - box_size/2), box_size, box_size, 
                        linewidth=2, edgecolor=color, facecolor='none'
                    )
                    ax.add_patch(rect)
                    
                    # 정점 이름 표시
                    ax.text(info['lon'], info['lat'] + box_size/2, info['station'], 
                            fontsize=8, color='black', ha='center', va='bottom',
                            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
                
                # 범례 요소 생성
                low_patch = patches.Rectangle((0, 0), 1, 1, linewidth=2, edgecolor=cm(0), facecolor='none')
                med_patch = patches.Rectangle((0, 0), 1, 1, linewidth=2, edgecolor=cm(0.5), facecolor='none')
                high_patch = patches.Rectangle((0, 0), 1, 1, linewidth=2, edgecolor=cm(1), facecolor='none')
                
                # 범례 추가
                ax.legend(
                    [low_patch, med_patch, high_patch], 
                    [f'적은 쓰레기 ({min_debris}개)', f'중간 쓰레기', f'많은 쓰레기 ({max_debris}개)'], 
                    loc='upper left',
                    fontsize=10
                )
            
            # 제목 및 레이블
            ax.set_title(f'선박밀집도 ({date.strftime("%Y-%m-%d")}) - 네모: 해양쓰레기 정점 (색상: 쓰레기 양)', fontsize=14)
            ax.set_xlabel('경도', fontsize=12)
            ax.set_ylabel('위도', fontsize=12)
            
            # 컬러바 추가
            if i == 0:
                global cbar
                cbar = fig.colorbar(scatter)
                cbar.set_label('선박밀집도 (%)')
            
            # 한국 해안선 형태를 대략적으로 표현하기 위한 좌표 범위 설정
            ax.set_xlim([126, 130])
            ax.set_ylim([34, 38])
            
            # 그리드 추가
            ax.grid(True, linestyle='--', alpha=0.6)
            
            # 현재 표시된 정점 수를 화면에 표시
            if debris_info:
                ax.text(
                    0.02, 0.97, 
                    f'표시된 해양쓰레기 정점: {len(debris_info)}개',
                    transform=ax.transAxes,
                    fontsize=12,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
                )
            
        except Exception as e:
            print(f"애니메이션 프레임 {i} 처리 오류: {str(e)}")
    
    # 애니메이션 생성
    ani = animation.FuncAnimation(fig, animate, frames=len(file_paths), interval=1000, blit=False)
    
    # GIF로 저장
    writer = animation.PillowWriter(fps=1)
    ani.save(output_gif, writer=writer)
    
    print(f"애니메이션을 {output_gif}에 저장했습니다.")
    plt.close()

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 경로 설정
    base_path = 'D:/marideb/code/sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    station_grid_file = 'station_nearest_grid.csv'  # 정점별 가장 가까운 그리드 정보 파일
    debris_data_file = 'D:/marideb/code/marideb_location.csv'  # 해양쓰레기 데이터 파일
    
    # 선박밀집도 애니메이션 생성 (7일간)
    create_ship_density_animation(base_path, station_grid_file, debris_data_file, num_days=7)

7개 날짜의 데이터를 애니메이션으로 만듭니다.
해양쓰레기 정점 그리드 60개를 로드했습니다.
해양쓰레기 데이터 로드: 개수 범위 45~11420
애니메이션을 ship_density_animation_JAN_grad.gif에 저장했습니다.


매 00시가 아니라 매시간의 추이를 봐야함. 그래서 매시간 자료를 같이 이어붙여서 한번더 봐야지 ㅠ

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap, Normalize
import os
import glob
from datetime import datetime, timedelta
import matplotlib.patches as patches

def create_ship_density_animation(base_path, station_grid_file, debris_data_file, output_gif='ship_density_animation_24_janweek1.gif', num_days=3):
    """
    여러 날짜와 시간의 선박밀집도 데이터를 애니메이션으로 시각화하고
    해양쓰레기 정점을 쓰레기 양에 따라 색상이 다른 네모로 표시
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - station_grid_file: 정점별 가장 가까운 그리드 정보 파일 경로
    - debris_data_file: 해양쓰레기 데이터 파일 경로
    - output_gif: 출력 GIF 파일 경로
    - num_days: 애니메이션에 포함할 날짜 수 (너무 많으면 파일 크기 커짐)
    """
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 날짜 및 시간 리스트 생성
    file_paths = []
    
    # 모든 시간대 포맷 (00시부터 23시까지)
    hours = [f"{h:02d}00" for h in range(24)]
    
    print(f"{num_days}일간, 24시간 데이터를 애니메이션으로 만듭니다.")
    
    # 날짜와 시간별 파일 경로 생성
    for day in range(num_days):
        date = start_date + timedelta(days=day)
        date_formatted = date.strftime('%Y%m%d')
        
        for hour in hours:
            file_pattern = os.path.join(base_path, f"sden_{date_formatted}{hour}_grid3.csv")
            
            if os.path.exists(file_pattern):
                # 날짜와 시간 정보 저장
                datetime_obj = datetime.strptime(f"{date_formatted}{hour}", '%Y%m%d%H%M')
                file_paths.append((datetime_obj, file_pattern))
            else:
                print(f"파일이 존재하지 않습니다: {file_pattern}")
    
    if not file_paths:
        print("애니메이션으로 만들 데이터 파일을 찾을 수 없습니다.")
        return
    
    # 시간 순으로 정렬
    file_paths.sort(key=lambda x: x[0])
    
    print(f"총 {len(file_paths)}개 시간대의 데이터를 애니메이션으로 만듭니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    # 정점별 가장 가까운 그리드 정보 로드
    station_grids = {}
    if os.path.exists(station_grid_file):
        station_grid_data = pd.read_csv(station_grid_file)
        for _, row in station_grid_data.iterrows():
            station_grids[row['정점명']] = row['가장_가까운_그리드']
        print(f"해양쓰레기 정점 그리드 {len(station_grids)}개를 로드했습니다.")
    else:
        print(f"경고: 정점 그리드 파일이 존재하지 않습니다: {station_grid_file}")
    
    # 해양쓰레기 데이터 로드
    debris_data = {}
    if os.path.exists(debris_data_file):
        debris_df = pd.read_csv(debris_data_file)
        
        # 개수 또는 무게 정보가 있는지 확인
        if '개수' in debris_df.columns:
            # 정점별 쓰레기 개수 저장
            for _, row in debris_df.iterrows():
                debris_data[row['정점명']] = {
                    '개수': row['개수'],
                    '무게_kg': row['무게_kg'] if '무게_kg' in debris_df.columns else 0
                }
            
            # 개수 범위 설정
            min_debris = debris_df['개수'].min()
            max_debris = debris_df['개수'].max()
            
            print(f"해양쓰레기 데이터 로드: 개수 범위 {min_debris}~{max_debris}")
        else:
            print("해양쓰레기 데이터에 '개수' 컬럼이 없습니다.")
    else:
        print(f"경고: 해양쓰레기 데이터 파일이 존재하지 않습니다: {debris_data_file}")
    
    # 모든 데이터에서 밀집도 최대값 찾기 (색상 스케일 통일을 위해)
    max_density = 0
    
    for _, file_path in file_paths:
        data = pd.read_csv(file_path)
        max_value = data['밀집도(%)'].max()
        if max_value > max_density:
            max_density = max_value
    
    # 애니메이션 설정
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 해양쓰레기 양에 따른 색상 정규화
    if debris_data:
        debris_amounts = [data['개수'] for data in debris_data.values()]
        min_debris = min(debris_amounts)
        max_debris = max(debris_amounts)
        debris_norm = Normalize(vmin=min_debris, vmax=max_debris)
    
    # 애니메이션 함수
    def animate(i):
        ax.clear()
        datetime_obj, file_path = file_paths[i]
        
        try:
            data = pd.read_csv(file_path)
            
            # 산점도 그리기
            x = []  # 경도
            y = []  # 위도
            s = []  # 마커 크기 (교통량)
            c = []  # 색상 (밀집도)
            
            # 해양쓰레기 정점 그리드 위치 및 색상 정보
            debris_info = []
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    if grid_id in grid_locations:
                        lat, lon = grid_locations[grid_id]
                        y.append(lat)
                        x.append(lon)
                        s.append(max(traffic / 5, 10))  # 교통량에 따른 마커 크기 조정, 최소 크기 10
                        c.append(density)
                        
                        # 이 그리드에 해당하는 정점 찾기
                        for station, station_grid in station_grids.items():
                            if station_grid == grid_id:
                                # 해양쓰레기 데이터가 있는 경우
                                if station in debris_data:
                                    debris_count = debris_data[station]['개수']
                                    debris_weight = debris_data[station]['무게_kg']
                                    
                                    # 해양쓰레기 정보 저장
                                    debris_info.append({
                                        'station': station,
                                        'grid': grid_id,
                                        'lon': lon,
                                        'lat': lat,
                                        'count': debris_count,
                                        'weight': debris_weight
                                    })
                except:
                    continue
            
            # 일반 그리드 데이터 그리기
            scatter = ax.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                                 vmin=0, vmax=max_density, edgecolors='k', linewidths=0.5)
            
            # 해양쓰레기 정점 그리드 표시 (쓰레기 양에 따른 색상)
            box_size = 0.08  # 네모 크기 조정
            
            if debris_info:
                # 범례를 위한 패치 리스트
                legend_elements = []
                
                for info in debris_info:
                    # 쓰레기 양에 따라 색상 결정
                    if 'count' in info:
                        color = cm(debris_norm(info['count']))
                    else:
                        color = 'lime'  # 기본 색상
                    
                    # 네모 그리기
                    rect = patches.Rectangle(
                        (info['lon'] - box_size/2, info['lat'] - box_size/2), box_size, box_size, 
                        linewidth=2, edgecolor=color, facecolor='none'
                    )
                    ax.add_patch(rect)
                    
                    # 정점 이름 표시
                    ax.text(info['lon'], info['lat'] + box_size/2, info['station'], 
                            fontsize=8, color='black', ha='center', va='bottom',
                            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
                
                # 범례 요소 생성
                low_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=cm(0), facecolor='none')
                med_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=cm(0.5), facecolor='none')
                high_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=cm(1), facecolor='none')
                
                # 범례 추가
                ax.legend(
                    [low_patch, med_patch, high_patch], 
                    [f'적은 쓰레기 ({min_debris}개)', f'중간 쓰레기', f'많은 쓰레기 ({max_debris}개)'], 
                    loc='upper left',
                    fontsize=10
                )
            
            # 제목 및 레이블
            ax.set_title(f'선박밀집도 ({datetime_obj.strftime("%Y-%m-%d %H:%M")}) - 네모: 해양쓰레기 정점 (색상: 쓰레기 양)', fontsize=14)
            ax.set_xlabel('경도', fontsize=12)
            ax.set_ylabel('위도', fontsize=12)
            
            # 컬러바 추가
            if i == 0:
                global cbar
                cbar = fig.colorbar(scatter)
                cbar.set_label('선박밀집도 (%)')
            
            # 한국 해안선 형태를 대략적으로 표현하기 위한 좌표 범위 설정
            ax.set_xlim([126, 130])
            ax.set_ylim([34, 38])
            
            # 그리드 추가
            ax.grid(True, linestyle='--', alpha=0.6)
            
            # 현재 표시된 정점 수와 진행 상황을 화면에 표시
            if debris_info:
                ax.text(
                    0.02, 0.97, 
                    f'해양쓰레기 정점: {len(debris_info)}개 | 프레임: {i+1}/{len(file_paths)}',
                    transform=ax.transAxes,
                    fontsize=12,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
                )
            
        except Exception as e:
            print(f"애니메이션 프레임 {i} 처리 오류: {str(e)}")
    
    # 애니메이션 생성
    ani = animation.FuncAnimation(fig, animate, frames=len(file_paths), interval=300, blit=False)
    
    # GIF로 저장 (dpi를 낮추어 파일 크기 감소)
    writer = animation.PillowWriter(fps=4)
    ani.save(output_gif, writer=writer, dpi=100)
    
    print(f"애니메이션을 {output_gif}에 저장했습니다.")
    plt.close()

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    station_grid_file = 'station_nearest_grid.csv'  # 정점별 가장 가까운 그리드 정보 파일
    debris_data_file = r'D:\marideb\code\marideb_location.csv'  # 해양쓰레기 데이터 파일
    
    # 선박밀집도 애니메이션 생성 (기본 3일간의 24시간 데이터)
    create_ship_density_animation(base_path, station_grid_file, debris_data_file, num_days=7)

7일간, 24시간 데이터를 애니메이션으로 만듭니다.
총 168개 시간대의 데이터를 애니메이션으로 만듭니다.
해양쓰레기 정점 그리드 60개를 로드했습니다.
해양쓰레기 데이터 로드: 개수 범위 45~11420
애니메이션을 ship_density_animation_24_janweek1.gif에 저장했습니다.


해류의 영향까지 같이 봐야 할 것 같다. 

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap, Normalize
import os
import glob
from datetime import datetime, timedelta
import matplotlib.patches as patches

def create_ship_density_animation(base_path, station_grid_file, debris_data_file, output_gif='ship_density_animation_24_janweek1.gif', num_days=3):
    """
    여러 날짜와 시간의 선박밀집도 데이터를 애니메이션으로 시각화하고
    해양쓰레기 정점을 쓰레기 양에 따라 색상이 다른 네모로 표시
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - station_grid_file: 정점별 가장 가까운 그리드 정보 파일 경로
    - debris_data_file: 해양쓰레기 데이터 파일 경로
    - output_gif: 출력 GIF 파일 경로
    - num_days: 애니메이션에 포함할 날짜 수 (너무 많으면 파일 크기 커짐)
    """
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 날짜 및 시간 리스트 생성
    file_paths = []
    
    # 모든 시간대 포맷 (00시부터 23시까지)
    hours = [f"{h:02d}00" for h in range(24)]
    
    print(f"{num_days}일간, 24시간 데이터를 애니메이션으로 만듭니다.")
    
    # 날짜와 시간별 파일 경로 생성
    for day in range(num_days):
        date = start_date + timedelta(days=day)
        date_formatted = date.strftime('%Y%m%d')
        
        for hour in hours:
            file_pattern = os.path.join(base_path, f"sden_{date_formatted}{hour}_grid3.csv")
            
            if os.path.exists(file_pattern):
                # 날짜와 시간 정보 저장
                datetime_obj = datetime.strptime(f"{date_formatted}{hour}", '%Y%m%d%H%M')
                file_paths.append((datetime_obj, file_pattern))
            else:
                print(f"파일이 존재하지 않습니다: {file_pattern}")
    
    if not file_paths:
        print("애니메이션으로 만들 데이터 파일을 찾을 수 없습니다.")
        return
    
    # 시간 순으로 정렬
    file_paths.sort(key=lambda x: x[0])
    
    print(f"총 {len(file_paths)}개 시간대의 데이터를 애니메이션으로 만듭니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    # 정점별 가장 가까운 그리드 정보 로드
    station_grids = {}
    if os.path.exists(station_grid_file):
        station_grid_data = pd.read_csv(station_grid_file)
        for _, row in station_grid_data.iterrows():
            station_grids[row['정점명']] = row['가장_가까운_그리드']
        print(f"해양쓰레기 정점 그리드 {len(station_grids)}개를 로드했습니다.")
    else:
        print(f"경고: 정점 그리드 파일이 존재하지 않습니다: {station_grid_file}")
    
    # 해양쓰레기 데이터 로드
    debris_data = {}
    if os.path.exists(debris_data_file):
        debris_df = pd.read_csv(debris_data_file)
        
        # 개수 또는 무게 정보가 있는지 확인
        if '개수' in debris_df.columns:
            # 정점별 쓰레기 개수 저장
            for _, row in debris_df.iterrows():
                debris_data[row['정점명']] = {
                    '개수': row['개수'],
                    '무게_kg': row['무게_kg'] if '무게_kg' in debris_df.columns else 0
                }
            
            # 개수 범위 설정
            min_debris = debris_df['개수'].min()
            max_debris = debris_df['개수'].max()
            
            print(f"해양쓰레기 데이터 로드: 개수 범위 {min_debris}~{max_debris}")
        else:
            print("해양쓰레기 데이터에 '개수' 컬럼이 없습니다.")
    else:
        print(f"경고: 해양쓰레기 데이터 파일이 존재하지 않습니다: {debris_data_file}")
    
    # 모든 데이터에서 밀집도 최대값 찾기 (색상 스케일 통일을 위해)
    max_density = 0
    
    for _, file_path in file_paths:
        data = pd.read_csv(file_path)
        max_value = data['밀집도(%)'].max()
        if max_value > max_density:
            max_density = max_value
    
    # 애니메이션 설정
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 쓰레기 양에 따른 박스 색상 - 세 구간으로 명확하게 구분 (파랑, 녹색, 빨강)
    
    # 해양쓰레기 양에 따른 색상 정규화
    if debris_data:
        debris_amounts = [data['개수'] for data in debris_data.values()]
        min_debris = min(debris_amounts)
        max_debris = max(debris_amounts)
        debris_norm = Normalize(vmin=min_debris, vmax=max_debris)
    
    # 애니메이션 함수
    def animate(i):
        ax.clear()
        datetime_obj, file_path = file_paths[i]
        
        try:
            data = pd.read_csv(file_path)
            
            # 산점도 그리기
            x = []  # 경도
            y = []  # 위도
            s = []  # 마커 크기 (교통량)
            c = []  # 색상 (밀집도)
            
            # 해양쓰레기 정점 그리드 위치 및 색상 정보
            debris_info = []
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    if grid_id in grid_locations:
                        lat, lon = grid_locations[grid_id]
                        y.append(lat)
                        x.append(lon)
                        s.append(max(traffic / 5, 10))  # 교통량에 따른 마커 크기 조정, 최소 크기 10
                        c.append(density)
                        
                        # 이 그리드에 해당하는 정점 찾기
                        for station, station_grid in station_grids.items():
                            if station_grid == grid_id:
                                # 해양쓰레기 데이터가 있는 경우
                                if station in debris_data:
                                    debris_count = debris_data[station]['개수']
                                    debris_weight = debris_data[station]['무게_kg']
                                    
                                    # 해양쓰레기 정보 저장
                                    debris_info.append({
                                        'station': station,
                                        'grid': grid_id,
                                        'lon': lon,
                                        'lat': lat,
                                        'count': debris_count,
                                        'weight': debris_weight
                                    })
                except:
                    continue
            
            # 일반 그리드 데이터 그리기
            scatter = ax.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                                 vmin=0, vmax=max_density, edgecolors='k', linewidths=0.5)
            
            # 해양쓰레기 정점 그리드 표시 (쓰레기 양에 따른 색상)
            box_size = 0.08  # 네모 크기 조정
            
            if debris_info:
                # 범례를 위한 패치 리스트
                legend_elements = []
                
                # 실제 화면에 표시되는 정점들의 쓰레기 양을 기준으로 3분할
                if debris_info:
                    # 표시되는 정점들의 쓰레기 양 목록 추출
                    displayed_debris_amounts = [info['count'] for info in debris_info if 'count' in info]
                    
                    if displayed_debris_amounts:
                        # 정렬하여 33퍼센타일과 66퍼센타일 지점 찾기
                        sorted_amounts = sorted(displayed_debris_amounts)
                        third_index = len(sorted_amounts) // 3
                        two_thirds_index = 2 * third_index
                        
                        # 균등하게 3분할하는 임계값 설정
                        low_threshold = sorted_amounts[third_index] if third_index < len(sorted_amounts) else min(sorted_amounts)
                        high_threshold = sorted_amounts[two_thirds_index] if two_thirds_index < len(sorted_amounts) else max(sorted_amounts)
                        
                        # 각 카테고리에 몇 개의 정점이 포함되는지 계산
                        low_count = sum(1 for val in displayed_debris_amounts if val <= low_threshold)
                        medium_count = sum(1 for val in displayed_debris_amounts if low_threshold < val <= high_threshold)
                        high_count = sum(1 for val in displayed_debris_amounts if val > high_threshold)
                        
                        print(f"쓰레기 분포: 적음({low_count}개), 중간({medium_count}개), 많음({high_count}개)")
                    else:
                        # 기본값 설정
                        low_threshold = 0
                        high_threshold = 0
                
                for info in debris_info:
                    # 쓰레기 양에 따라 색상 결정 (파랑: 적음, 녹색: 중간, 빨강: 많음)
                    if 'count' in info:
                        count = info['count']
                        
                        # 명확한 구분을 위해 구간별로 색상 지정
                        if count <= low_threshold:
                            color = (0, 0, 1)  # 파랑 (적은 쓰레기)
                            face_color = (0, 0, 1, 0.3)
                        elif count <= high_threshold:
                            color = (0, 0.8, 0)  # 녹색 (중간 쓰레기)
                            face_color = (0, 0.8, 0, 0.3)
                        else:
                            color = (1, 0, 0)  # 빨강 (많은 쓰레기)
                            face_color = (1, 0, 0, 0.3)
                    else:
                        color = 'blue'  # 기본 색상
                        face_color = (0, 0, 1, 0.3)  # 기본 색상 (투명도 0.3)
                    
                    # 네모 그리기 (테두리와 동일한 색상으로 채움)
                    rect = patches.Rectangle(
                        (info['lon'] - box_size/2, info['lat'] - box_size/2), box_size, box_size, 
                        linewidth=2, edgecolor=color, facecolor=face_color
                    )
                    ax.add_patch(rect)
                    
                    # 정점 이름 표시
                    ax.text(info['lon'], info['lat'] + box_size/2, info['station'], 
                            fontsize=8, color='black', ha='center', va='bottom',
                            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
                
                # 범례 요소 생성 (파랑, 녹색, 빨강)
                low_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=(0, 0, 1), facecolor=(0, 0, 1, 0.3))
                med_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=(0, 0.8, 0), facecolor=(0, 0.8, 0, 0.3))
                high_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=(1, 0, 0), facecolor=(1, 0, 0, 0.3))
                
                # 범례 추가 (균등 분할된 정점 수 표시)
                ax.legend(
                    [low_patch, med_patch, high_patch], 
                    [f'적은 쓰레기 (≤{low_threshold}개, {low_count}개 정점)', 
                     f'중간 쓰레기 ({low_threshold}~{high_threshold}개, {medium_count}개 정점)', 
                     f'많은 쓰레기 (>{high_threshold}개, {high_count}개 정점)'], 
                    loc='upper left',
                    fontsize=10
                )
            
            # 제목 및 레이블
            ax.set_title(f'선박밀집도 ({datetime_obj.strftime("%Y-%m-%d %H:%M")}) - 네모: 해양쓰레기 정점 (파랑: 적음, 녹색: 중간, 빨강: 많음)', fontsize=14)
            ax.set_xlabel('경도', fontsize=12)
            ax.set_ylabel('위도', fontsize=12)
            
            # 컬러바 추가
            if i == 0:
                global cbar
                cbar = fig.colorbar(scatter)
                cbar.set_label('선박밀집도 (%)')
            
            # 한국 해안선 형태를 대략적으로 표현하기 위한 좌표 범위 설정
            ax.set_xlim([126, 130])
            ax.set_ylim([34, 38])
            
            # 그리드 추가
            ax.grid(True, linestyle='--', alpha=0.6)
            
            # 현재 표시된 정점 수와 진행 상황을 화면에 표시
            if debris_info:
                ax.text(
                    0.02, 0.97, 
                    f'해양쓰레기 정점: {len(debris_info)}개 | 프레임: {i+1}/{len(file_paths)}',
                    transform=ax.transAxes,
                    fontsize=12,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
                )
            
        except Exception as e:
            print(f"애니메이션 프레임 {i} 처리 오류: {str(e)}")
    
    # 애니메이션 생성
    ani = animation.FuncAnimation(fig, animate, frames=len(file_paths), interval=300, blit=False)
    
    # GIF로 저장 (dpi를 낮추어 파일 크기 감소)
    writer = animation.PillowWriter(fps=4)
    ani.save(output_gif, writer=writer, dpi=100)
    
    print(f"애니메이션을 {output_gif}에 저장했습니다.")
    plt.close()

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    station_grid_file = 'station_nearest_grid.csv'  # 정점별 가장 가까운 그리드 정보 파일
    debris_data_file = r'D:\marideb\code\marideb_location.csv'  # 해양쓰레기 데이터 파일
    
    # 선박밀집도 애니메이션 생성 (기본 3일간의 24시간 데이터)
    create_ship_density_animation(base_path, station_grid_file, debris_data_file, num_days=30)

30일간, 24시간 데이터를 애니메이션으로 만듭니다.
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301130100_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301130200_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301130300_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301251300_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301251400_grid3.csv
총 715개 시간대의 데이터를 애니메이션으로 만듭니다.
해양쓰레기 정점 그리드 60개를 로드했습니다.
해양쓰레기 데이터 로드: 개수 범위 45~11420
쓰레기 분포: 적음(19개), 중간(18개), 많음(17개)
쓰레기 분포: 적음(19개), 중간(18개), 많음(17개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(17개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(16개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(16개)
쓰레기 분포: 적음(17개), 중간(16개), 많음(17개)
쓰레기 분포: 적음(17개), 중간(16개), 많음(16개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(17개)
쓰레기 분포: 적음(19개), 중간(18개), 많음(17개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(18개)
쓰레기 분포: 적음(19개), 중간(18개), 많음(17개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(18개)
쓰레기 분포: 적음(19개), 중간(18개), 많음(17개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(18개)
쓰레기 분포: 적음(18개), 중간(17개), 많음(18개)
쓰레기 분포: 적

해류

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime, timedelta
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap, Normalize

def analyze_cumulative_ship_density(base_path, station_grid_file, debris_data_file, output_file='cumulative_ship_density_vs_debris.png', num_days=30):
    """
    30일간의 누적 선박밀집도를 계산하고 해양쓰레기 양과 비교 분석
    
    Parameters:
    - base_path: 선박밀집도 데이터 파일이 있는 기본 경로
    - station_grid_file: 정점별 가장 가까운 그리드 정보 파일 경로
    - debris_data_file: 해양쓰레기 데이터 파일 경로
    - output_file: 출력 이미지 파일 경로
    - num_days: 누적할 일수 (기본 30일)
    """
    # 시작 날짜 설정 (예: 2023년 1월 1일)
    start_date = datetime(2023, 1, 1)
    
    # 모든 시간대 포맷 (00시부터 23시까지)
    hours = [f"{h:02d}00" for h in range(24)]
    
    print(f"{num_days}일간의 누적 선박밀집도를 계산합니다.")
    
    # 모든 파일 경로 수집
    file_paths = []
    
    for day in range(num_days):
        date = start_date + timedelta(days=day)
        date_formatted = date.strftime('%Y%m%d')
        
        for hour in hours:
            file_pattern = os.path.join(base_path, f"sden_{date_formatted}{hour}_grid3.csv")
            
            if os.path.exists(file_pattern):
                datetime_obj = datetime.strptime(f"{date_formatted}{hour}", '%Y%m%d%H%M')
                file_paths.append((datetime_obj, file_pattern))
            else:
                print(f"파일이 존재하지 않습니다: {file_pattern}")
    
    if not file_paths:
        print("데이터 파일을 찾을 수 없습니다.")
        return
    
    # 시간 순으로 정렬
    file_paths.sort(key=lambda x: x[0])
    
    print(f"총 {len(file_paths)}개 시간대의 데이터를 분석합니다.")
    
    # 첫 번째 파일로 그리드 위치 정보 추출
    grid_locations = {}
    
    first_data = pd.read_csv(file_paths[0][1])
    
    for _, row in first_data.iterrows():
        try:
            grid_id = row['격자번호']
            lat_lon = row['위경도'].split(', ')
            lat = float(lat_lon[0])
            lon = float(lat_lon[1])
            grid_locations[grid_id] = (lat, lon)
        except:
            continue
    
    # 정점별 가장 가까운 그리드 정보 로드
    station_grids = {}
    if os.path.exists(station_grid_file):
        station_grid_data = pd.read_csv(station_grid_file)
        for _, row in station_grid_data.iterrows():
            station_grids[row['정점명']] = row['가장_가까운_그리드']
        print(f"해양쓰레기 정점 그리드 {len(station_grids)}개를 로드했습니다.")
    else:
        print(f"경고: 정점 그리드 파일이 존재하지 않습니다: {station_grid_file}")
    
    # 해양쓰레기 데이터 로드
    debris_data = {}
    if os.path.exists(debris_data_file):
        debris_df = pd.read_csv(debris_data_file)
        
        # 개수 또는 무게 정보가 있는지 확인
        if '개수' in debris_df.columns:
            # 정점별 쓰레기 개수 저장
            for _, row in debris_df.iterrows():
                debris_data[row['정점명']] = {
                    '개수': row['개수'],
                    '무게_kg': row['무게_kg'] if '무게_kg' in debris_df.columns else 0
                }
            
            # 개수 범위 설정
            min_debris = debris_df['개수'].min()
            max_debris = debris_df['개수'].max()
            
            print(f"해양쓰레기 데이터 로드: 개수 범위 {min_debris}~{max_debris}")
        else:
            print("해양쓰레기 데이터에 '개수' 컬럼이 없습니다.")
    else:
        print(f"경고: 해양쓰레기 데이터 파일이 존재하지 않습니다: {debris_data_file}")
    
    # 누적 선박밀집도 저장 딕셔너리
    cumulative_density = {}
    
    # 모든 파일에서 선박밀집도 누적
    for i, (datetime_obj, file_path) in enumerate(file_paths):
        if i % 24 == 0:  # 하루에 24개 시간대가 있으므로 진행상황 표시
            print(f"처리 중: {datetime_obj.strftime('%Y-%m-%d %H:%M')} ({i+1}/{len(file_paths)})")
            
        try:
            data = pd.read_csv(file_path)
            
            for _, row in data.iterrows():
                try:
                    grid_id = row['격자번호']
                    density = row['밀집도(%)']
                    traffic = row['교통량(척)']
                    
                    # 누적 선박밀집도 및 교통량 계산
                    if grid_id not in cumulative_density:
                        cumulative_density[grid_id] = {
                            '누적_밀집도': 0,
                            '누적_교통량': 0,
                            '데이터_수': 0
                        }
                    
                    cumulative_density[grid_id]['누적_밀집도'] += density
                    cumulative_density[grid_id]['누적_교통량'] += traffic
                    cumulative_density[grid_id]['데이터_수'] += 1
                except:
                    continue
        except Exception as e:
            print(f"파일 처리 오류: {file_path}, 오류: {str(e)}")
    
    # 평균 선박밀집도 계산
    for grid_id, data in cumulative_density.items():
        if data['데이터_수'] > 0:
            data['평균_밀집도'] = data['누적_밀집도'] / data['데이터_수']
            data['평균_교통량'] = data['누적_교통량'] / data['데이터_수']
        else:
            data['평균_밀집도'] = 0
            data['평균_교통량'] = 0
    
    # 최대 누적 밀집도 찾기
    max_cumulative_density = max([data['누적_밀집도'] for data in cumulative_density.values()])
    
    # 시각화
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # 커스텀 컬러맵 생성
    colors = [(0, 0, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0)]  # 파랑, 초록, 노랑, 빨강
    cmap_name = 'ship_density_cmap'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    # 해양쓰레기 양에 따른 색상 정의 (파랑, 녹색, 빨강)
    debris_colors = {
        'low': (0, 0, 1),     # 파랑
        'medium': (0, 0.8, 0), # 녹색
        'high': (1, 0, 0)      # 빨강
    }
    
    # 산점도 그리기
    x = []  # 경도
    y = []  # 위도
    s = []  # 마커 크기 (평균 교통량)
    c = []  # 색상 (누적 밀집도)
    
    # 해양쓰레기 정점 정보
    debris_info = []
    
    for grid_id, data in cumulative_density.items():
        if grid_id in grid_locations:
            lat, lon = grid_locations[grid_id]
            y.append(lat)
            x.append(lon)
            s.append(max(data['평균_교통량'] / 5, 10))  # 평균 교통량에 따른 마커 크기 조정, 최소 크기 10
            c.append(data['누적_밀집도'])
            
            # 이 그리드에 해당하는 정점 찾기
            for station, station_grid in station_grids.items():
                if station_grid == grid_id and station in debris_data:
                    debris_count = debris_data[station]['개수']
                    debris_weight = debris_data[station]['무게_kg']
                    
                    # 해양쓰레기 정보 저장
                    debris_info.append({
                        'station': station,
                        'grid': grid_id,
                        'lon': lon,
                        'lat': lat,
                        'count': debris_count,
                        'weight': debris_weight,
                        'density': data['누적_밀집도'],
                        'traffic': data['누적_교통량']
                    })
    
    # 일반 그리드 데이터 그리기
    scatter = ax.scatter(x, y, s=s, c=c, cmap=cm, alpha=0.7, 
                         vmin=0, vmax=max_cumulative_density, edgecolors='k', linewidths=0.5)
    
    # 해양쓰레기 정점 그리드 표시
    if debris_info:
        # 표시되는 정점들의 쓰레기 양 목록 추출
        displayed_debris_amounts = [info['count'] for info in debris_info]
        
        if displayed_debris_amounts:
            # 정렬하여 33퍼센타일과 66퍼센타일 지점 찾기
            sorted_amounts = sorted(displayed_debris_amounts)
            third_index = len(sorted_amounts) // 3
            two_thirds_index = 2 * third_index
            
            # 균등하게 3분할하는 임계값 설정
            low_threshold = sorted_amounts[third_index] if third_index < len(sorted_amounts) else min(sorted_amounts)
            high_threshold = sorted_amounts[two_thirds_index] if two_thirds_index < len(sorted_amounts) else max(sorted_amounts)
            
            # 각 카테고리에 몇 개의 정점이 포함되는지 계산
            low_count = sum(1 for val in displayed_debris_amounts if val <= low_threshold)
            medium_count = sum(1 for val in displayed_debris_amounts if low_threshold < val <= high_threshold)
            high_count = sum(1 for val in displayed_debris_amounts if val > high_threshold)
            
            print(f"쓰레기 분포: 적음({low_count}개), 중간({medium_count}개), 많음({high_count}개)")
    
    # 박스 크기 설정
    box_size = 0.08
    
    # 정점 그리기
    for info in debris_info:
        # 쓰레기 양에 따라 색상 결정
        if info['count'] <= low_threshold:
            color = debris_colors['low']  # 파랑 (적은 쓰레기)
            face_color = (*color, 0.3)
            category = 'low'
        elif info['count'] <= high_threshold:
            color = debris_colors['medium']  # 녹색 (중간 쓰레기)
            face_color = (*color, 0.3)
            category = 'medium'
        else:
            color = debris_colors['high']  # 빨강 (많은 쓰레기)
            face_color = (*color, 0.3)
            category = 'high'
        
        # 네모 그리기
        rect = patches.Rectangle(
            (info['lon'] - box_size/2, info['lat'] - box_size/2), box_size, box_size, 
            linewidth=2, edgecolor=color, facecolor=face_color
        )
        ax.add_patch(rect)
        
        # 정점 이름 표시
        ax.text(info['lon'], info['lat'] + box_size/2, info['station'], 
                fontsize=8, color='black', ha='center', va='bottom',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    # 범례 요소 생성
    low_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=debris_colors['low'], facecolor=(*debris_colors['low'], 0.3))
    med_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=debris_colors['medium'], facecolor=(*debris_colors['medium'], 0.3))
    high_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor=debris_colors['high'], facecolor=(*debris_colors['high'], 0.3))
    
    # 범례 추가
    ax.legend(
        [low_patch, med_patch, high_patch], 
        [f'적은 쓰레기 (≤{low_threshold}개, {low_count}개 정점)', 
         f'중간 쓰레기 ({low_threshold}~{high_threshold}개, {medium_count}개 정점)', 
         f'많은 쓰레기 (>{high_threshold}개, {high_count}개 정점)'], 
        loc='upper left',
        fontsize=10
    )
    
    # 제목 및 레이블
    ax.set_title(f'{num_days}일간 누적 선박밀집도와 해양쓰레기 비교 ({start_date.strftime("%Y-%m-%d")}부터)', fontsize=14)
    ax.set_xlabel('경도', fontsize=12)
    ax.set_ylabel('위도', fontsize=12)
    
    # 컬러바 추가
    cbar = fig.colorbar(scatter)
    cbar.set_label('누적 선박밀집도')
    
    # 한국 해안선 형태를 대략적으로 표현하기 위한 좌표 범위 설정
    ax.set_xlim([126, 130])
    ax.set_ylim([34, 38])
    
    # 그리드 추가
    ax.grid(True, linestyle='--', alpha=0.6)
    
    # 상관관계 분석
    if debris_info:
        # 정점별 누적 선박밀집도와 쓰레기 양 데이터 추출
        density_values = [info['density'] for info in debris_info]
        debris_values = [info['count'] for info in debris_info]
        
        # 상관계수 계산
        correlation = np.corrcoef(density_values, debris_values)[0, 1]
        
        # 상관관계 정보 추가
        ax.text(
            0.02, 0.05, 
            f'선박밀집도-쓰레기 양 상관계수: {correlation:.3f}',
            transform=ax.transAxes,
            fontsize=12,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
        )
        
        # 산점도 추가 (작은 서브플롯)
        ax_inset = fig.add_axes([0.68, 0.05, 0.25, 0.25])
        ax_inset.scatter(density_values, debris_values, c=[debris_colors[('low' if val <= low_threshold else 'medium' if val <= high_threshold else 'high')] for val in debris_values], alpha=0.7)
        ax_inset.set_xlabel('누적 선박밀집도')
        ax_inset.set_ylabel('쓰레기 개수')
        ax_inset.set_title('선박밀집도 vs 쓰레기 양')
        ax_inset.grid(True, linestyle='--', alpha=0.6)
        
        # 정점 개수 및 분석 정보 표시
        ax.text(
            0.02, 0.97, 
            f'해양쓰레기 정점: {len(debris_info)}개 | 분석 기간: {num_days}일',
            transform=ax.transAxes,
            fontsize=12,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7)
        )
    
    # 이미지 저장
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    print(f"누적 선박밀집도와 해양쓰레기 비교 이미지를 {output_file}에 저장했습니다.")
    
    # 추가 분석: 정점별 선박밀집도와 쓰레기 양 테이블 생성
    if debris_info:
        analysis_data = []
        
        for info in debris_info:
            analysis_data.append({
                '정점명': info['station'],
                '위도': info['lat'],
                '경도': info['lon'],
                '쓰레기_개수': info['count'],
                '쓰레기_무게_kg': info['weight'],
                '누적_선박밀집도': info['density'],
                '누적_교통량': info['traffic'],
                '쓰레기_카테고리': 'low' if info['count'] <= low_threshold else 'medium' if info['count'] <= high_threshold else 'high'
            })
        
        # 데이터프레임 생성 및 CSV 저장
        analysis_df = pd.DataFrame(analysis_data)
        
        # 상관관계 분석
        corr_density_count = analysis_df['누적_선박밀집도'].corr(analysis_df['쓰레기_개수'])
        corr_traffic_count = analysis_df['누적_교통량'].corr(analysis_df['쓰레기_개수'])
        
        print(f"누적 선박밀집도와 쓰레기 개수의 상관계수: {corr_density_count:.3f}")
        print(f"누적 교통량과 쓰레기 개수의 상관계수: {corr_traffic_count:.3f}")
        
        # 분석 결과 저장
        analysis_file = output_file.replace('.png', '_analysis.csv')
        analysis_df.to_csv(analysis_file, index=False, encoding='utf-8-sig')
        print(f"정점별 분석 결과를 {analysis_file}에 저장했습니다.")
    
    plt.close()
    return

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 경로 설정
    base_path = r'D:\marideb\code\sden_2023_lv3'  # 선박밀집도 데이터 기본 경로
    station_grid_file = 'station_nearest_grid.csv'  # 정점별 가장 가까운 그리드 정보 파일
    debris_data_file = r'D:\marideb\code\marideb_location.csv'  # 해양쓰레기 데이터 파일
    
    # 30일간 누적 선박밀집도 분석
    analyze_cumulative_ship_density(base_path, station_grid_file, debris_data_file, num_days=30)

30일간의 누적 선박밀집도를 계산합니다.
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301130100_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301130200_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301130300_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301251300_grid3.csv
파일이 존재하지 않습니다: D:\marideb\code\sden_2023_lv3\sden_202301251400_grid3.csv
총 715개 시간대의 데이터를 분석합니다.
해양쓰레기 정점 그리드 60개를 로드했습니다.
해양쓰레기 데이터 로드: 개수 범위 45~11420
처리 중: 2023-01-01 00:00 (1/715)
처리 중: 2023-01-02 00:00 (25/715)
처리 중: 2023-01-03 00:00 (49/715)
처리 중: 2023-01-04 00:00 (73/715)
처리 중: 2023-01-05 00:00 (97/715)
처리 중: 2023-01-06 00:00 (121/715)
처리 중: 2023-01-07 00:00 (145/715)
처리 중: 2023-01-08 00:00 (169/715)
처리 중: 2023-01-09 00:00 (193/715)
처리 중: 2023-01-10 00:00 (217/715)
처리 중: 2023-01-11 00:00 (241/715)
처리 중: 2023-01-12 00:00 (265/715)
처리 중: 2023-01-13 00:00 (289/715)
처리 중: 2023-01-14 03:00 (313/715)
처리 중: 2023-01-15 03:00 (337/715)
처리 중: 2023-01-16 03:00 (361/715)
처리 중: 2023-

C:\Users\TAN\AppData\Local\Temp\ipykernel_13156\1834149770.py:323: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


누적 선박밀집도와 해양쓰레기 비교 이미지를 cumulative_ship_density_vs_debris.png에 저장했습니다.
누적 선박밀집도와 쓰레기 개수의 상관계수: 0.100
누적 교통량과 쓰레기 개수의 상관계수: 0.069
정점별 분석 결과를 cumulative_ship_density_vs_debris_analysis.csv에 저장했습니다.
